In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb

from modules.featureEng import *
from modules.modelEng import *

# to display entire data
#pd.set_option('display.max_rows', 1500)
#pd.set_option('display.max_columns', 150)

/Users/necronia/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
"""
Load Data
"""

train_df = pd.read_csv('data/train.csv', index_col='Id')
test_df = pd.read_csv('data/test.csv', index_col='Id')

In [3]:
"""
Feature Engineering
"""

train_X_bf = train_df.loc[:,train_df.columns!='SalePrice']
train_Y_bf = train_df['SalePrice']
test_X_bf = test_df

### for testing ###
#from sklearn.model_selection import train_test_split
#train_X_bf, test_X_bf, train_Y_bf, test_Y = train_test_split(train_X_bf, train_Y_bf, random_state=777)
###################

train_X, train_Y, test_X = pre_processing(train_X_bf.copy(), train_Y_bf.copy(), test_X_bf.copy())

In [4]:
"""
Predict house price
"""

elastic_net = grid_search(train_X, train_Y, ElasticNet())
lasso = grid_search(train_X, train_Y, Lasso())
ridge = grid_search(train_X, train_Y, Ridge())
#random_forest = grid_search(train_X, train_Y, RandomForestRegressor())
support_vector_regressor = grid_search(train_X, train_Y, SVR())
XGBoost = grid_search(train_X, train_Y, xgb.XGBRegressor())
#light_GBM = grid_search(train_X, train_Y, lgb.LGBMRegressor())

stacked_regression = StackingRegressor(
        #regressors=[elastic_net, lasso, ridge, random_forest, XGBoost, light_GBM],
        regressors=[elastic_net, lasso, ridge, support_vector_regressor, XGBoost],
        meta_regressor=SVR(kernel='rbf')
)

stacked_regression.fit(train_X, train_Y)

stacked = stacked_regression.predict(test_X)

ensembled = np.expm1((0.2 * elastic_net.predict(test_X)) +
                     (0.2 * lasso.predict(test_X)) +
                     (0.1 * ridge.predict(test_X)) +
                     #(0.05 * random_forest.predict(test_X)) +
                     (0.2 * support_vector_regressor.predict(test_X)) +
                     (0.1 * XGBoost.predict(test_X)) +
                     #(0.2 * light_GBM.predict(test_X)) +
                     (0.2 * stacked))
                     

stacked_regression.score(train_X, train_Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(15,12))
t1 = sorted(np.expm1(train_Y))
t1 = pd.DataFrame(t1)
plt.scatter(t1.index, t1,c='r')

result = pd.read_csv('data/submission.csv', index_col='Id')
t2 = sorted(result.SalePrice)
t2 = pd.DataFrame(t2)
plt.scatter(t2.index, t2,c='b')

In [ ]:
### for testing ###
#RMSE = np.mean((ensembled - test_Y)**2)**(1/2) 
#print('Score : ' + str(RMSE))
###################

"""
Export submission data
"""
submission = pd.DataFrame({
    'Id':test_X.index + (len(train_X_bf) - len(train_X) + 1),
    'SalePrice':ensembled
})
submission.to_csv('data/submission.csv', index=False)

In [ ]:
#train_X.hist(bins=50, figsize=(15,10))

In [ ]:
train_X